# Applied Data Science Capstone Project - IBM

## Segmenting and Clustering Neighborhoods in Toronto, part 3

In [4]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
!pip install folium
#Folium did not load so it had to be installed

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 5.9 MB/s  eta 0:00:01


In [5]:
import folium

On the beggining we load Dataframe that we saved in part 2

In [6]:
df=pd.read_csv('Capstone2.csv')
df.head()

,Unnamed: 0_x,Postal Code,Borough_x,Neighborhood_x,Latitude,Longitude,Unnamed: 0_y,Borough_y,Neighborhood_y
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,0,North York,Parkwoods
1,1,M4A,North York,Victoria Village,43.725882,-79.315572,1,North York,Victoria Village
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Downtown Toronto,"Regent Park, Harbourfront"
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3,North York,"Lawrence Manor, Lawrence Heights"
4,4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,4,Queen's Park,Ontario Provincial Government


In [11]:
df=df[df['Borough_x'].str.contains('Toronto')]
df=df.reset_index(drop=True)
df.head()

,Unnamed: 0_x,Postal Code,Borough_x,Neighborhood_x,Latitude,Longitude,Unnamed: 0_y,Borough_y,Neighborhood_y
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Downtown Toronto,"Regent Park, Harbourfront"
1,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,9,Downtown Toronto,"Garden District, Ryerson"
2,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,15,Downtown Toronto,St. James Town
3,19,M4E,East Toronto,The Beaches,43.676357,-79.293031,19,East Toronto,The Beaches
4,20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,20,Downtown Toronto,Berczy Park


In [13]:
df['Borough_x'].value_counts()

Downtown Toronto          17
Central Toronto            9
West Toronto               6
East Toronto               4
East Toronto Business      1
Downtown Toronto Stn A     1
East York/East Toronto     1
Name: Borough_x, dtype: int64

Looks like data is not consistent so we have to clean it

In [25]:
df1 = df.replace({'East Toronto Business':'East Toronto', 'Downtown Toronto Stn A':'Downtown Toronto','East York/East Toronto':'East Toronto'})
df2 = df1.drop(['Unnamed: 0_x', 'Unnamed: 0_x','Unnamed: 0_y','Borough_y','Neighborhood_y'], axis=1)
df2.head()

,Postal Code,Borough_x,Neighborhood_x,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [26]:
df2['Borough_x'].value_counts()

Downtown Toronto    18
Central Toronto      9
East Toronto         6
West Toronto         6
Name: Borough_x, dtype: int64

Lets add Groups for each neighborhood

In [27]:
df2['Group']=df2['Borough_x'].replace(to_replace=['Downtown Toronto','Central Toronto','West Toronto','East Toronto'],value=[1,2,3,4],inplace=False)
df2.head()

,Postal Code,Borough_x,Neighborhood_x,Latitude,Longitude,Group
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,4
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1


In [28]:
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [38]:
kclusters=len(df2.Group.unique())
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, cluster in zip(df2['Latitude'], df2['Longitude'], df2['Group']):
    Group = folium.Popup(str(df2['Borough_x']) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=Group,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_toronto)

Finally, lets print the map:

In [40]:
map_toronto